## False-color UPenn image block

**Input**
- nucdir (str)  : Directory of the Ch1 image block ( 2D .JPEG stacks), eg. "test/Ch1/block1"
- cytodir (str) : Directory of the Ch2 image block ( 2D .JPEG stacks), eg. "test/Ch1/block2"
- FChome (str)  : Directory of the resulting FC block to be saved , .eg "test"

**Output**
- Create a folder under *"FChome/FC/blockname"*
- Save the false-colored 2D .JPEG stacks in the folder created.

In [1]:
import os
import numpy as np
from skimage.io import imread, imsave
from Functions.fcoloring import *

In [4]:
# Change these 3
nucdir = r"W:\ITAS3D_training_data\PGP9.5\validation_UPenn_AFM045\test\ch1\OTLS4_NODO_4-4-23_AFM045_well_1_Xindex_000003_Yindex_000000"
cytodir = r"W:\ITAS3D_training_data\PGP9.5\validation_UPenn_AFM045\test\ch2\OTLS4_NODO_4-4-23_AFM045_well_1_Xindex_000003_Yindex_000000"
FC_home = r"W:\ITAS3D_training_data\PGP9.5\validation_UPenn_AFM045\test"

HE_settings = {'nuclei': [0.17, 0.27, 0.105], 'cyto': [0.05, 1.0, 0.54]}
nuc_normfactor=3000
cyto_normfactor=8000

nuc_block = os.listdir(nucdir)
cyto_block = os.listdir(cytodir)
nuc_block_path = [os.path.join(nucdir, i) for i in nuc_block]
cyto_block_path = [os.path.join(cytodir, i) for i in cyto_block]

blockname = cytodir.split("OTLS4")[1]
FC_folder = FC_home + os.sep + "FC"
FC_block_dir = FC_folder + os.sep + blockname
if not os.path.exists(FC_folder):
    os.mkdir(FC_folder)
if not os.path.exists(FC_block_dir):
    os.mkdir(FC_block_dir)

FC_blocks = [os.path.join(FC_block_dir, i ) for i in nuc_block]

for nuc, cyto, FC in zip(nuc_block_path, cyto_block_path, FC_blocks):

    nuc  = imread(nuc)
    cyto = imread(cyto)
    nuc =  FC_rescale(nuc,  1, 10000)
    cyto = FC_rescale(cyto, 1, 10000)

    pseudoHE = rapidFalseColor(nuc, cyto, 
                               HE_settings['nuclei'], HE_settings['cyto'],
                               nuc_normfactor=nuc_normfactor, cyto_normfactor=cyto_normfactor)
    imsave(FC, pseudoHE)